In [ ]:
#Question 3: Transformer-Based Sentiment Analysis Web App (Hard)

pip install torch transformers datasets scikit-learn streamlit


SyntaxError: invalid syntax (ipython-input-1-3125302169.py, line 3)

In [ ]:
%pip install torch transformers datasets scikit-learn streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd

# Load your TSV
df = pd.read_csv('drugLibTrain_raw.tsv', sep='\t')

# Create sentiment label
df['sentiment'] = df['rating'].apply(lambda x: 1 if x >= 7 else 0)

# Only keep review text and label
df = df[['benefitsReview', 'sentiment']].dropna()

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['benefitsReview'], truncation=True, padding=True, max_length=128)

from datasets import Dataset

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Train-test split
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/3089 [00:00<?, ? examples/s]

In [ ]:
# Load dataset
df = pd.read_csv("drugLibTrain_raw.tsv", sep="\t")
# Use a small subset for fast training
df = df.sample(n=50, random_state=42)
# Use 'benefitsReview' and 'sideEffectsReview' as reviews and 'rating' for sentiment
df = df.dropna(subset=["benefitsReview", "sideEffectsReview", "rating"])


In [ ]:
# Combine the review columns into a single 'review' column
df['review'] = df['benefitsReview'].fillna('') + ' ' + df['sideEffectsReview'].fillna('')


In [ ]:
# Binary sentiment: rating ≥ 7 is positive (1), else negative (0)
df["label"] = df["rating"].apply(lambda x: 1 if x >= 7 else 0)


In [ ]:
# Take text and labels
texts = df["review"].tolist()
labels = df["label"].tolist()


In [ ]:
metrics = trainer.evaluate()
print(metrics)



Epoch,Training Loss,Validation Loss
0,No log,No log


{}


In [ ]:
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")


('./sentiment_model/tokenizer_config.json',
 './sentiment_model/special_tokens_map.json',
 './sentiment_model/vocab.txt',
 './sentiment_model/added_tokens.json')

In [ ]:
import streamlit as st
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load
tokenizer = DistilBertTokenizer.from_pretrained('./sentiment_model')
model = DistilBertForSequenceClassification.from_pretrained('./sentiment_model')

st.title("💊 Drug Review Sentiment Analyzer (Transformers)")

text = st.text_area("Enter a drug review:")

if st.button("Analyze Sentiment"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    sentiment = torch.argmax(probs).item()
    confidence = probs[0][sentiment].item()
    label = "Positive" if sentiment == 1 else "Negative"
    st.write(f"**Prediction:** {label}")
    st.write(f"**Confidence:** {confidence:.2f}")


2025-07-11 06:50:30.736 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-11 06:50:30.894 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-11 06:50:30.896 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-11 06:50:30.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-11 06:50:30.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-11 06:50:30.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-11 06:50:30.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-11 06:50:30.907 Thread 'MainThread': mi

In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.91.253:8501

3f07264353d851c357417bbc06e6888b5e4dfebd
  Stopping...
  Stopping...
^C
